---
title: APIs
subtitle: "Application Programming Interfaces"
author:
  - name: Charles Pletcher
    affiliations: Tufts University
    orcid: 0000-0003-2734-5511
    email: charles.pletcher@tufts.edu
license:
  code: MIT
date: 2025-03-31
---

# Application Programming Interfaces

You will hear the term "API" in many different contexts when it comes to computer programming. Already in this class, we have discussed Python's API for the file system.

Often, however, when someone mentions an API, they are referring to a web-based API that is usually accessed over HTTP(S). You might have heard about the kerfuffle when Twitter shut down much of the access to its API, or when Reddit did the same thing a few years earlier. These APIs are servers that provide _interfaces_ (the "I" in "API") to a platform's data.

As you probably noticed while reading @Walker2019, it is not exactly uncommon for references to APIs to become out of date.

Luckily, we can still use the API provided by the [Digital Public Library of America](https://dp.la) for our work for this class.

We'll be working with the Python [Requests](https://docs.python-requests.org/en/latest/) library, which provides its own easy-to-use API for making HTTP requests. In other words, it's APIs all the way down.

## Getting an access token

Generally, APIs will ask that you first obtain a key to use them. Even if APIs offer unlimited requests, it is important for them to require users to supply an API key so that they can track (often anonymized) usage statistics, errors, and so on.

Sometimes, APIs require you to pay, either immediately or after making a certain number of requests. Keys can be used to track usage for payment calculations, too. For an example of this system, see OpenAI's [pricing page](https://openai.com/api/pricing/).

### An API Key for DPLA

For this tutorial, we'll work with the Digital Public Library of America's (DPLA) API. Take a few minutes to read through their [API Basics](https://pro.dp.la/developers/api-basics), then request an API key.

:::{note} Request types

You'll notice that you must submit a `POST` request to receive an API key. `POST` is one of several HTTP verbs. When you enter a URL into a web browser and hit "Enter," you're typically issuing a `GET` request: `GET` requests do not have a request body; they simply ask for the information at the provided URL, perhaps with some query parameters (the `key=value` pairs after a `?` in the URL).

`POST` requests, by contrast, _may_ contain a request body. You've probably submitted `POST` requests without knowing it whenever you sign up for a new service. That's essentially what we're doing with DPLA here, we're just doing it from the command line instead of through an interface that DPLA has built.
:::

The DPLA [documentation](https://pro.dp.la/developers/policies#get-a-key) instructs you to submit a request using `curl`, but we don't have access to `curl` from this notebook. Instead, let's make the request using the Python "Requests" library.

In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests

my_email = ""

requests.post(f"https://api.dp.la/v2/api_key/{my_email}")

<Response [200]>

After running the above code cell, you should receive an email with your API code. It's good practice not to share these codes or include them in version control (i.e., git).

Instead, create an account-specific [secret](https://docs.github.com/en/codespaces/managing-your-codespaces/managing-your-account-specific-secrets-for-github-codespaces) by following the instructions provided by GitHub. 

Let's call the secret `DPLA_API_KEY`. (It's conventional to use all caps for environment variables and secrets.)

Make sure to give your fork of this repository access to the secret, and then restart this codespace. We'll be here when you get back.

## Making your first request

As we saw above, making requests using the `requests` library is pretty straightforward — for a `GET` request, we can just pass a URL to `requests.get()`.

In order for the request to be successful, though, we'll need to include the API key in the `api_key` querystring parameter. And to do that, we'll need to use the `os` library in Python.

In [1]:
import os
import requests

DPLA_API_KEY = os.getenv("DPLA_API_KEY")

Let's use the example provided by the DPLA documentation, querying for the term "weasels".

In [3]:

requests.get(f"https://api.dp.la/v2/items?q=weasels&api_key={DPLA_API_KEY}")


<Response [200]>

`<Response [200]>` means that our request was successful, but it doesn't give us a whole lot of information. This is because we have not read the response body. To do so, let's assign the response — which is the return value of `requests.get()` — to a variable and read it as JSON.

In [5]:
response = requests.get(f"https://api.dp.la/v2/items?q=weasels&api_key={DPLA_API_KEY}")

response.json()['docs'][1]

{'@context': 'http://dp.la/api/items/context',
 '@id': 'http://dp.la/api/items/1bb63407b0c8a0b59958cba153269f81',
 '@type': 'ore:Aggregation',
 'aggregatedCHO': '#sourceResource',
 'dataProvider': {'@id': 'http://dp.la/api/contributor/the-miriam-and-ira-d-wallach-division-of-art-prints-and-photographs-picture-collection-the-new-york-public-library',
  'name': 'The Miriam and Ira D. Wallach Division of Art, Prints and Photographs: Picture Collection. The New York Public Library'},
 'id': '1bb63407b0c8a0b59958cba153269f81',
 'ingestDate': '2025-03-01T19:25:34.013Z',
 'ingestType': 'item',
 'isShownAt': 'https://digitalcollections.nypl.org/items/8c93ee70-c536-012f-8c06-58d385a7bc34',
 'object': 'https://images.nypl.org/index.php?t=t&id=823782',
 'originalRecord': {'stringValue': '{\n  "id" : 3098486,\n  "type" : "Item",\n  "uuid" : "8c93ee70-c536-012f-8c06-58d385a7bc34",\n  "thumbnail_url" : "https://images.nypl.org/index.php?t=t&id=823782",\n  "capture_image_ids" : [ "823782" ],\n  "crea

## Reading responses

As you can see above, the response returns a JSON (JavaScript Object Notation) object with a few top-level keys. If you're thinking, "Hm, this JSON looks an awful lot like a Python dictionary," you're absolutely right. While the semantics of Python dictionaries and JSON _are_ different, in this case, the `requests` library has already coerced the raw JSON to a Python dictionary for us. You can access its values like you would with any Python dict:

In [6]:
parsed_response = response.json()

parsed_response['count']

245

:::{important}
Experiment a bit. How, for example, would you get all of the titles in a list?
:::

## Constructing queries

Naturally, when you're working with an API, you'll want to be able to construct your own queries. Above, we hard-coded the value `weasels` under the querystring parameter `q`. But you can use Python's string interpolation to set any value you want. For example

In [7]:
my_query = "foxes"
my_url = f"https://api.dp.la/v2/items?q={my_query}&api_key={DPLA_API_KEY}"

response = requests.get(my_url)

parsed_response = response.json()
parsed_response

{'count': 1580,
 'docs': [{'@context': 'http://dp.la/api/items/context',
   '@id': 'http://dp.la/api/items/d5728fb0ffcd131d28c33cbefd7437d0',
   '@type': 'ore:Aggregation',
   'aggregatedCHO': '#sourceResource',
   'dataProvider': {'@id': 'http://dp.la/api/contributor/denver-public-library',
    'exactMatch': ['http://www.wikidata.org/entity/Q5259775'],
    'name': 'Denver Public Library'},
   'id': 'd5728fb0ffcd131d28c33cbefd7437d0',
   'ingestDate': '2025-01-29T20:03:33.435Z',
   'ingestType': 'item',
   'isShownAt': 'https://digital.denverlibrary.org/nodes/view/1050323',
   'object': 'https://digital.denverlibrary.org/assets/nodeimg/1050323',
   'originalRecord': {'stringValue': '<record \nxmlns="http://www.openarchives.org/OAI/2.0/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n  <header>\n    <identifier>\n      oai:YOUR_OAI_PREFIX:DPL:oai:digital.denverlibrary.org:1050323\n    </identifier>\n    <datestamp>2025-01-27T21:49:44Z</datestamp>\n  </header>\n  <metadata>\n   

You could even write a function that puts constructs the request URL and returns the parsed response so that you don't have to do these things manually over and over again.

In [8]:
def make_dpla_request(query: str):
    url = f"https://api.dp.la/v2/items?q={query.replace(" ","+")}&api_key={DPLA_API_KEY}"
    response = requests.get(url)

    return response.json()

In [9]:
make_dpla_request("red foxes")

{'count': 138,
 'docs': [{'@context': 'http://dp.la/api/items/context',
   '@id': 'http://dp.la/api/items/d9dd4dc15414b61176ea5400aae31089',
   '@type': 'ore:Aggregation',
   'aggregatedCHO': '#sourceResource',
   'dataProvider': {'@id': 'http://dp.la/api/contributor/missouri-state-archives-through-missouri-digital-heritage',
    'name': 'Missouri State Archives through Missouri Digital Heritage'},
   'id': 'd9dd4dc15414b61176ea5400aae31089',
   'iiifManifest': 'http://cdm16795.contentdm.oclc.org/iiif/info/p16795coll24/4146/manifest.json',
   'ingestDate': '2025-01-02T21:39:04.590Z',
   'ingestType': 'item',
   'isShownAt': 'http://cdm16795.contentdm.oclc.org/cdm/ref/collection/p16795coll24/id/4146',
   'object': 'http://cdm16795.contentdm.oclc.org/utils/getthumbnail/collection/p16795coll24/id/4146',
   'originalRecord': {'stringValue': '{\n  "@context" : "http://dp.la/api/items/context",\n  "isShownAt" : "http://cdm16795.contentdm.oclc.org/cdm/ref/collection/p16795coll24/id/4146",\n  

There's a problem with this code, however. What happens if you try to make a request with a query that contains spaces, such as `"red foxes"`?

Can you find the appropriate workaround using the documentation? https://pro.dp.la/developers/requests

What other features does this API support?

## RESTful APIs

Many APIs, including the DPLA's, are built on RESTful principles. REST stands for **Re**presentational **S**tate **T**ransfer. In terms of web APIs, REST means that a given server will respond with a representation of the data that it has available, and that representation will contain additional information for manipulating the data or requesting further data.

Although it is not, strictly speaking, a requirement of REST APIs, many REST implementations use a predictable URL scheme.

For example, you might find a list of "collections" at the `/collections` endpoint. To request a specific collection, you would append its ID — e.g., for Collection 3, `/collections/3`.

Each collection might contain items, so to get a list of items in Collection 3 you could send a request to `/collections/3/items`. And then to get a specific item in that collection — you guessed it, `/collections/3/items/12`.

DPLA does _not_ implement this kind of schema, and instead relies on facets and other search parameters. But it is worth being aware of such schemes if you want to use other APIs in your work and research.

## Readings

- @Walker2019
- @Matthes2023 [chs. 15–17]

## Homework

Design and test an experiment using the data from a publicly available API, such as the [Digital Public Library of America](https://pro.dp.la/developers) or [Chronicling America](https://chroniclingamerica.loc.gov/about/api/) — you can also use another data source, just run it by me first.

In your report, be sure to discuss your research question, hypothesis, methods, results, and conclusion — in other words, walk the reader through the full scientific process.

These experiments need not be large — think of a small, answerable question that you could tackle in the space of 4 hours of work (i.e., the amount of outside work generally expected for each lab).